In [22]:
import geopandas as gpd
import json
import pandas as pd
from geojson import dump

In [23]:
sf = gpd.read_file("SAL_2021_AUST_GDA2020_SHP/SAL_2021_AUST_GDA2020.shp")
sf = sf.rename(columns = {'SAL_NAME21': 'suburb_name'})
sf=sf.dropna(axis=0,subset=['geometry'])
sf=sf[sf['STE_NAME21'] == 'Victoria']
sf

,SAL_CODE21,suburb_name,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,SHAPE_Leng,SHAPE_Area,geometry
4544,20001,Abbeyard,2,Victoria,AUS,Australia,327.5008,http://linked.data.gov.au/dataset/asgsed3/SAL/...,1.207678,0.033162,"POLYGON ((146.89824 -37.04602, 146.89947 -37.0..."
4545,20002,Abbotsford (Vic.),2,Victoria,AUS,Australia,1.7405,http://linked.data.gov.au/dataset/asgsed3/SAL/...,0.092990,0.000178,"POLYGON ((145.00195 -37.79665, 145.00190 -37.7..."
4546,20003,Aberfeldie,2,Victoria,AUS,Australia,1.5515,http://linked.data.gov.au/dataset/asgsed3/SAL/...,0.059374,0.000159,"POLYGON ((144.89576 -37.76514, 144.89547 -37.7..."
4547,20004,Aberfeldy,2,Victoria,AUS,Australia,10.8319,http://linked.data.gov.au/dataset/asgsed3/SAL/...,0.183199,0.001107,"POLYGON ((146.38814 -37.72232, 146.38808 -37.7..."
4548,20005,Acheron,2,Victoria,AUS,Australia,72.6602,http://linked.data.gov.au/dataset/asgsed3/SAL/...,0.514040,0.007381,"POLYGON ((145.76731 -37.25433, 145.76757 -37.2..."
...,...,...,...,...,...,...,...,...,...,...,...
7483,22940,Yundool,2,Victoria,AUS,Australia,31.6396,http://linked.data.gov.au/dataset/asgsed3/SAL/...,0.256626,0.003174,"POLYGON ((145.86040 -36.28432, 145.86038 -36.2..."
7484,22941,Yuroke,2,Victoria,AUS,Australia,8.8851,http://linked.data.gov.au/dataset/asgsed3/SAL/...,0.158951,0.000906,"POLYGON ((144.85250 -37.55800, 144.85303 -37.5..."
7485,22942,Yuulong,2,Victoria,AUS,Australia,52.1657,http://linked.data.gov.au/dataset/asgsed3/SAL/...,0.447416,0.005404,"POLYGON ((143.32185 -38.68969, 143.32203 -38.6..."
7486,22943,Zeerust,2,Victoria,AUS,Australia,18.0294,http://linked.data.gov.au/dataset/asgsed3/SAL/...,0.211342,0.001808,"POLYGON ((145.40454 -36.25294, 145.40479 -36.2..."


In [24]:
with open('city_count.json', 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data)
df['suburb_name']=None
for i in range(df.shape[0]):
    df.iloc[i,-1]=df.iloc[i,0].split(',')[0]
df=df.drop(1)
df


,full_name,count,suburb_name
0,"Melbourne, Victoria",2969,Melbourne
2,"Mirboo North, Victoria",3,Mirboo North
3,"Torquay - Jan Juc, Victoria",16,Torquay - Jan Juc
4,"Geelong, Victoria",59,Geelong
5,"Little River, Victoria",7,Little River
...,...,...,...
90,"Hamilton, Victoria",1,Hamilton
91,"Echuca, Victoria",4,Echuca
92,"Teesdale, Victoria",1,Teesdale
93,"Officer, Victoria",1,Officer


In [29]:
gdf = gpd.GeoDataFrame(
    pd.merge(df, sf, on = 'suburb_name', how = 'right')
)
gdf['count'] = gdf['count'].fillna(0)
non_0_gdf=gdf[gdf['count']!=0.0]
non_0_gdf['count'].describe().to_frame()

,count
count,67.000000
mean,48.850746
std,362.248209
min,1.000000
25%,1.000000
50%,1.000000
75%,5.500000
max,2969.000000


In [26]:
geoJSON = gdf[['suburb_name', 'geometry','count']].to_json()
geoJSON = json.loads(geoJSON)

In [27]:
with open('city_geo_data.json','w') as f:
    json.dump(geoJSON,f)